In [ ]:
from gurobipy import *
import numpy as np
import time
import random
import pandas as pd
import random
import scipy.sparse as sp
import gurobipy as gp
from gurobipy import GRB
from itertools import combinations, permutations, product

#dist = np.loadtxt("stuff_distanse.txt")

class Problem(object):
    def __init__(self,
                 number_of_order=0,
                 number_of_sku=0,
                 number_of_unique_sku=0,
                 number_of_batch=0):
        self.number_of_order = number_of_order
        self.number_of_sku = number_of_sku
        self.number_of_unique_sku = number_of_unique_sku
        self.number_of_batch = number_of_batch

    def To_Original(self):
        self.list_of_unique_sku = np.load(r'./Data/order.npy',
                                          allow_pickle=True)
        self.Ki = np.load(r'./Data/Ki.npy', allow_pickle=True)
        self.Ik = np.load(r'./Data/Ik.npy', allow_pickle=True)
        self.Ib = np.load(r'./Data/Ib.npy', allow_pickle=True)
        self.lib = np.load(r'./Data/lib.npy', allow_pickle=True)
        self.rib = np.load(r'./Data/rib.npy', allow_pickle=True)
        self.mb = np.load(r'./Data/mb.npy', allow_pickle=True)
        self.a = np.load(r'./Data/a.npy', allow_pickle=True)
        self.skubin = np.load("./Data/skubin.npy")
        self.Bi = [([], i) for i in range(len(s.Ki))]
        for i in range(len(self.Ki)):
            for j in range(len(self.Ib)):
                if i in self.Ib[j]:
                    self.Bi[i][0].append(j)

        length = [[], []]
        for i in range(len(self.Bi)):
            length[0].append(len(self.Bi[i][0]))
            length[1].append(i)

        length = np.array(length)

        length = length.T[np.lexsort(length[::-1, :])].T

        Bi_temp = [([], length[1][i]) for i in range(len(self.Bi))]
        for i in range(len(self.Bi)):
            Bi_temp[i][0].extend(self.Bi[length[1][i]][0])
        self.Bi = Bi_temp

        c = 1
        while True:
            if c > max(length[0]):
                break
            elif not np.any(np.where(length[0] == c)[0]):
                c += 1
                pass
            else:
                left = np.where(np.array(length[0]) == c)[0][0]
                right = np.where(np.array(length[0]) == c)[0][-1] + 1
                self.Bi[left:right] = sorted(self.Bi[left:right])
                c += 1

        self.Pi = []
        temp = [self.Bi[0][1]]
        c = 0
        for i in range(1, len(self.Bi)):
            if self.Bi[i][0] == self.Bi[i - 1][0]:
                temp.append(self.Bi[i][1])
            else:
                self.Pi.append((c, temp, self.Bi[i - 1][0]))
                temp = [self.Bi[i][1]]
                c += 1
        self.Pi.append(
            (c, temp, self.Bi[i][0]))  #第一位是 Pattern 序号  第二位是订单序号  第三位是架子集合序号
        #Pb={p\ 第b个货架上有商品在p中}
        self.Pb = [[] for i in range(len(self.mb))]
        for i in range(len(self.Pi)):
            for j in self.Pi[i][2]:
                self.Pb[j].append(self.Pi[i][0])
        # Qp[i]=len{Pi第i行的订单个数}
        self.Qp = []
        for i in self.Pi:
            self.Qp.append(len(i[1]))

        #List_M
        self.Set_List_M = set()
        for i in self.Pi[::-1]:
            self.Set_List_M = self.Set_List_M | set(i[1])
            if len(self.Set_List_M) > 50:
                break



        self.number_of_order = len(self.Ki)
        #self.number_of_sku
        self.number_of_unique_sku = len(self.list_of_unique_sku)
        self.number_of_batch = 100


    def oib(self, i, b):
        # i 订单编号
        # b 货架编号
        # 目标：输出货架靠左的货架编号
        m = s.mb[b]
        for k in s.Ki[i]:
            l = s.skubin[s.list_of_unique_sku[k]]
            if l[0] - 1 == b:
                if l[1] < m:
                    m = l[1]
        return m

    def dib(self, i, b):
        # i 订单编号
        # b 货架编号
        # 目标：输出货架靠右的货架编号

        m = 1
        for k in s.Ki[i]:
            l = s.skubin[s.list_of_unique_sku[k]]
            if l[0] - 1 == b:
                if l[1] > m:
                    m = l[1]
        return m

    def form_data(self):
        #导入 商品-位置 文件
        self.skubin = np.load("./Data/skubin.npy")

        #导入 每个货架长度 文件
        self.mb = np.load(r'./Data/mb.npy', allow_pickle=True)

        #所有商品总数
        total_number_of_sku = len(self.skubin)

        #不重复商品集合通过0,1,2,...,total_number_of_sku-1 中取number_of_unique_sku个数随机生成
        self.list_of_unique_sku = random.sample(range(total_number_of_sku),
                                                self.number_of_unique_sku)
        self.list_of_unique_sku.sort()

        #稀疏矩阵和Ki初始化
        row = []
        col = []
        self.Ki = []

        #分配给每个订单多少商品量，几个球的隔板问题
        if self.number_of_order == 1:
            list_of_number_of_sku_in_order = [self.number_of_sku]
        else:
            while True:
                list_of_number_of_sku_in_order = []
                L = random.sample(range(1, self.number_of_sku),
                                  self.number_of_order - 1)
                L.sort()
                list_of_number_of_sku_in_order.append(L[0])
                for i in range(1, len(L)):
                    list_of_number_of_sku_in_order.append(L[i] - L[i - 1])
                list_of_number_of_sku_in_order.append(self.number_of_sku -
                                                      L[-1])

                if self.number_of_unique_sku >= max(
                        list_of_number_of_sku_in_order):
                    break

        #随机把商品分配给订单
        count = 0
        for i in list_of_number_of_sku_in_order:
            order_sku = random.sample(range(len(self.list_of_unique_sku)), i)
            self.Ki.append(order_sku)
            row.extend([count for j in range(i)])
            col.extend(order_sku)
            count = count + 1

        #矩阵a:第j(列)个商品在第i(行)个订单中的0-1矩阵
        self.a = sp.csr_matrix(
            (np.array([1 for i in range(len(row))]),
             (np.array(row), np.array(col))),
            shape=(self.number_of_order, len(self.list_of_unique_sku)))

        #Ik={i\ 第k个商品在i中}
        y = range(self.number_of_order)
        self.Ik = []
        for j in range(len(self.list_of_unique_sku)):
            self.Ik.append(
                list(
                    filter(lambda x: x != None, [
                        x if y else None
                        for x, y in zip(y, list(self.a.toarray()[:, j]))
                    ])))

        #Ib={i\ 第b个货架上有商品在i中}
        self.Ib = [[] for i in range(len(self.mb))]
        for i in range(len(self.Ik)):
            self.Ib[int(self.skubin[int(self.list_of_unique_sku[i])][0]) -
                    1].extend(self.Ik[i])

        # Bi={b| i要经过b货架}
        self.Bi = [([], i) for i in range(len(s.Ki))]
        for i in range(len(self.Ki)):
            for j in range(len(self.Ib)):
                if i in self.Ib[j]:
                    self.Bi[i][0].append(j)

        length = [[], []]
        for i in range(len(self.Bi)):
            length[0].append(len(self.Bi[i][0]))
            length[1].append(i)

        length = np.array(length)

        length = length.T[np.lexsort(length[::-1, :])].T

        Bi_temp = [([], length[1][i]) for i in range(len(self.Bi))]
        for i in range(len(self.Bi)):
            Bi_temp[i][0].extend(self.Bi[length[1][i]][0])
        self.Bi = Bi_temp

        c = 1
        while True:
            if c > max(length[0]):
                break
            elif not np.any(np.where(length[0] == c)[0]):
                c += 1
                pass
            else:
                left = np.where(np.array(length[0]) == c)[0][0]
                right = np.where(np.array(length[0]) == c)[0][-1] + 1
                self.Bi[left:right] = sorted(self.Bi[left:right])
                c += 1

        self.Pi = []
        temp = [self.Bi[0][1]]
        c = 0
        for i in range(1, len(self.Bi)):
            if self.Bi[i][0] == self.Bi[i - 1][0]:
                temp.append(self.Bi[i][1])
            else:
                self.Pi.append((c, temp, self.Bi[i - 1][0]))
                temp = [self.Bi[i][1]]
                c += 1
        self.Pi.append(
            (c, temp, self.Bi[i][0]))  #第一位是 Pattern 序号  第二位是订单序号  第三位是架子集合序号
        #Pb={p\ 第b个货架上有商品在p中}
        self.Pb = [[] for i in range(len(self.mb))]
        for i in range(len(self.Pi)):
            for j in self.Pi[i][2]:
                self.Pb[j].append(self.Pi[i][0])
        # Qp[i]=len{Pi第i行的订单个数}
        self.Qp = []
        for i in self.Pi:
            self.Qp.append(len(i[1]))

        #List_M

        self.Set_List_M = set()
        for i in self.Pi[::-1]:
            self.Set_List_M = self.Set_List_M | set(i[1])
            if len(self.Set_List_M) > 50:
                break


        #lib =min{t\ (b,t)}
        self.lib = np.zeros([self.number_of_order, len(self.mb)])
        for i in range(self.number_of_order):
            for j in range(len(self.mb)):
                self.lib[i][j] = self.oib(i, j)
        #rib-max{t\(b,t)}
        self.rib = np.zeros([self.number_of_order, len(self.mb)])
        for i in range(self.number_of_order):
            for j in range(len(self.mb)):
                self.rib[i][j] = self.dib(i, j)

   